# CIFAR-10 Training and evaluation pipeline
Autor: **Andrea Incerti Delmonte**

Email: ** andrea.incertidelmonte@gmail.com**

This training pipeline is based on TF Estimator https://www.tensorflow.org/get_started/custom_estimators
1. Hyperparameters and estimators configurations 
2. Load dataset metadata
3. Input pipeline definition
4. CNN network definition
5. Custom TF Estimator
6. Model training and evaluation
7. Model evaluation on test set

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cPickle
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
TF_RECORDS_BASE_PATH = "./cifar-10_dataset/tf_records/"
TRAIN_TFRECORDS = TF_RECORDS_BASE_PATH + "train.tfrecords"
EVAL_TFRECORDS = TF_RECORDS_BASE_PATH + "eval.tfrecords"
TEST_TFRECORDS = TF_RECORDS_BASE_PATH + "test.tfrecords"
METADATA_PATH = "./cifar-10_dataset/cifar-10-batches-py/batches.meta"

IMG_HEIGHT = 32
IMG_WIDTH = 32
IMG_CHANNELS = 3
CLASS_NUMBER = 10

## 1. Hyperparameters and estimators configurations 

In [ ]:
class FLAGS():
  pass 

FLAGS.lenet_model_name = "lenet"
FLAGS.deep_lenet_model_name = "deep-lenet"

###############################
#                             #
# Choose which model to train #
#                             #
###############################

FLAGS.model_name = FLAGS.deep_lenet_model_name

FLAGS.batch_size = 128
FLAGS.learning_rate = 0.001
FLAGS.max_steps = 60000
FLAGS.eval_steps = 1000
FLAGS.throttle_secs = 2*60
#FLAGS.tf_random_seed = 19861102
FLAGS.save_checkpoints_steps = 2000
FLAGS.use_checkpoint = False
FLAGS.exported_model_name = "exported-{}".format(FLAGS.model_name)
FLAGS.keep_checkpoint_max = 20

## 2. Load Cifar10 metadata

In [ ]:
metadata_f = open(METADATA_PATH, 'rb')
metadata_dict = cPickle.load(metadata_f)
labels_LUT = metadata_dict['label_names']
print(labels_LUT)

## 3. TFRecords processing pipeline

### 3.1 From serialized record to image and label

In [ ]:
def record_parser(record):
    features = tf.parse_single_example(
        record,
        features={
          'image': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([], tf.int64),
        }
    )
    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.reshape(image, shape=(IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH))
    image = tf.transpose(image, [1, 2, 0])
    
    label = tf.cast(features['label'], tf.int32)
    
    return image, label

def augment(image, label):
    
    # Apply random crop
    image = tf.image.resize_image_with_crop_or_pad(
        image, IMG_HEIGHT + 4, IMG_WIDTH + 4)
    image = tf.random_crop(image, [IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
    
    # Apply vertical random split
    image = tf.image.random_flip_left_right(image)
    
    return image, label

def normalize(image, label):
    image = tf.image.per_image_standardization(image)
    
    return image, label

### 3.2 Training batch preparation

In [ ]:
def generate_train_input_fn(train_tfrecords, batch_size=1):
    def _train_input_fn():
        train_dataset = tf.data.TFRecordDataset(train_tfrecords)
        train_dataset = train_dataset.shuffle(buffer_size=(batch_size*2 + 1))
        train_dataset = train_dataset.map(record_parser)
        train_dataset = train_dataset.map(augment)
        train_dataset = train_dataset.map(normalize)        
        train_dataset = train_dataset.repeat()
        train_dataset = train_dataset.batch(batch_size)
        train_dataset = train_dataset.prefetch(batch_size*2)
        train_iterator = train_dataset.make_one_shot_iterator()
        images, labels = train_iterator.get_next()
        features = {'images': images}
        
        return features, labels
    return _train_input_fn

#### 3.2.1 Test training batch preparation

In [ ]:
generated_train_input_fn = generate_train_input_fn(TRAIN_TFRECORDS, FLAGS.batch_size)
images_tensor, labels_tensor = generated_train_input_fn()

with tf.Session() as sess:    
    images_batch_dict, labels_batch = sess.run([images_tensor, labels_tensor])
    images_batch = images_batch_dict["images"]
    print("Train images batch shape {}".format(images_batch.shape))
    print("Train labels batch shape {}".format(labels_batch.shape))

    print("Image batch type {}".format(type(images_batch[0][0][0][0])))
    print("Label batch type {}".format(type(labels_batch[0])))

for i in range(min([2, images_batch.shape[0]])):
    
    image = images_batch[i]

    # Check image shape
    assert image.shape == (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

    # Plot the image
    plt.title("Train image label {} class {}".format(labels_batch[i], labels_LUT[labels_batch[i]]))
    plt.imshow(image)
    plt.show()

### 3.3 Evaluation batch preparation

In [ ]:
def generate_eval_input_fn(eval_tfrecords, batch_size=1):
    def _eval_input_fn():
        eval_dataset = tf.data.TFRecordDataset(eval_tfrecords)
        eval_dataset = eval_dataset.map(record_parser)
        eval_dataset = eval_dataset.map(normalize)
        eval_dataset = eval_dataset.repeat()
        eval_dataset = eval_dataset.batch(batch_size)
        eval_dataset = eval_dataset.prefetch(batch_size*2)
        eval_iterator = eval_dataset.make_one_shot_iterator()
        images, labels = eval_iterator.get_next()
        features = {'images': images}
        
        return features, labels
    return _eval_input_fn

#### 3.3.1 Test evaluation batch preparation

In [ ]:
generated_eval_input_fn = generate_eval_input_fn(EVAL_TFRECORDS, FLAGS.batch_size)
images_tensor, labels_tensor = generated_eval_input_fn()

with tf.Session() as sess:    
    images_batch_dict, labels_batch = sess.run([images_tensor, labels_tensor])
    images_batch = images_batch_dict["images"]
    print("Eval images batch shape {}".format(images_batch.shape))
    print("Eval labels batch shape {}".format(labels_batch.shape))

    print("Eval batch type {}".format(type(images_batch[0][0][0][0])))
    print("Eval batch type {}".format(type(labels_batch[0])))

for i in range(min([2, images_batch.shape[0]])):
    
    image = images_batch[i]

    # Check image shape
    assert image.shape == (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

    # Plot the image
    plt.title("Eval image label {} class {}".format(labels_batch[i], labels_LUT[labels_batch[i]]))
    plt.imshow(image)
    plt.show()

## 4. CNN network definition

### 4.1 LeNet

In [ ]:
def init_weight_variable(shape, mu=0, sigma=0.1):
    """
    weight_variable generates a weight variable of a given shape.
    """
    initial = tf.truncated_normal(shape, mean=mu, stddev=sigma)
    return tf.Variable(initial)


def init_bias_variable(shape, value=0.1):
    """
    bias_variable generates a bias variable of a given shape.
    """
    initial = tf.constant(value=value, shape=shape)
    return tf.Variable(initial)

def LeNet(x):    
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    with tf.name_scope("conv1"):
        W_conv1 = init_weight_variable([5, 5, 3, 6])
        b_conv1 = init_bias_variable([6])
        c_conv1 = tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='VALID') + b_conv1
        h_conv1 = tf.nn.relu(c_conv1)
        print("h_conv1.shape {}".format(h_conv1.shape))
  
    # Layer 1: Max Pooling. Input = 28x28x6. Output = 14x14x6.
    with tf.name_scope("pool1"):
        h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        print("h_pool1.shape {}".format(h_pool1.shape))

    
    # Layer 2: Convolutional. Input = 14x14x6. Output = 10x10x16.
    with tf.name_scope("conv2"):
        W_conv2 = init_weight_variable([5, 5, 6, 16])
        b_conv2 = init_bias_variable([16])
        c_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='VALID') + b_conv2
        h_conv2 = tf.nn.relu(c_conv2)
        print("h_conv2.shape {}".format(h_conv2.shape))
    
    # Layer 2: Max Pooling. Input = 10x10x16. Output = 5x5x16.
    with tf.name_scope("pool2"):
        h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        print("h_pool2.shape {}".format(h_pool2.shape))
    
    # Flatten. Input = 5x5x16. Output = 400. 
    with tf.name_scope("fc0"):
        fc0 = tf.layers.flatten(h_pool2)
        print("flatten.shape {}".format(fc0.shape))
    
    # Layer 3: Fully connected. Input = 400. Output = 120. 
    with tf.name_scope("fc1"):
        W_fc1 = init_weight_variable([400, 120])
        b_fc1 = init_bias_variable([120])
        mm_fc1 = tf.matmul(fc0, W_fc1) + b_fc1
        h_fc1 = tf.nn.relu(mm_fc1)
        print("h_fc1.shape {}".format(h_fc1.shape))

    # Layer 4: Fully connected. Input = 120. Output = 84. 
    with tf.name_scope("fc2"):
        W_fc2 = init_weight_variable([120, 84])
        b_fc2 = init_bias_variable([84])
        mm_fc2 = tf.matmul(h_fc1, W_fc2) + b_fc2
        h_fc2 = tf.nn.relu(mm_fc2)
        print("h_fc2.shape {}".format(h_fc2.shape))
    
    # Layer 5: Fully connected. Input = 84. Output = 10. 
    with tf.name_scope("fc3"):
        W_fc3 = init_weight_variable([84, 10])
        b_fc3 = init_bias_variable([10])
        logits = tf.matmul(h_fc2, W_fc3) + b_fc3
        print("h_fc3.shape {}".format(logits.shape))
        
    return logits

### 4.1 Deep LeNet

In [ ]:
def DeepLeNet(images):
             
    # Layer 1: Convolutional. Input = 32x32x3. Output = 32x32x64.
    conv1 = tf.layers.conv2d(
      inputs=images, filters=64, kernel_size=[5, 5], padding='SAME',
      activation=tf.nn.relu, name='Conv1')
    print("conv1.shape {}".format(conv1.shape))
    
    # Layer 1: Max Pooling. Input = 32x32x64. Output = 15x15x64.
    pool1 = tf.layers.max_pooling2d(
      inputs=conv1, pool_size=[3, 3], strides=2, name='MaxPool1')
    norm1 = tf.nn.lrn(
      pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='Norm1')
    print("pool1.shape {}".format(pool1.shape))

    # Layer 2: Convolutional. Input = 15x15x64. Output = 15x15x64.                                                                                                                
    conv2 = tf.layers.conv2d(
      inputs=norm1, filters=64, kernel_size=[5, 5], padding='SAME',
      activation=tf.nn.relu, name='Conv2')
    print("conv2.shape {}".format(conv2.shape))
    norm2 = tf.nn.lrn(
      conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='Norm2')
    
    # Layer 2: Max Pooling. Input = 15x15x64. Output = 7x7x64.
    pool2 = tf.layers.max_pooling2d(
      inputs=norm2, pool_size=[3, 3], strides=2, name='Pool2')
    print("pool2.shape {}".format(pool2.shape))

    # Flatten. Input = 7x7x64. Output = 3136.                                                                                                                         
    shape = pool2.get_shape()
    pool2_ = tf.reshape(pool2, [-1, shape[1]*shape[2]*shape[3]])
    print("flatten.shape {}".format(pool2_.shape))

    # Layer 3: Fully connected. Input = 3136. Output = 384.                                                                                                                
    dense1 = tf.layers.dense(
      inputs=pool2_, units=384, activation=tf.nn.relu, name='FC1')
    print("dense1.shape {}".format(dense1.shape))
    
    # Layer 4: Fully connected. Input = 384. Output = 192.                                                                                                       
    dense2 = tf.layers.dense(
      inputs=dense1, units=192, activation=tf.nn.relu, name='FC2')
    print("dense2.shape {}".format(dense2.shape))

    # Layer 5: Fully connected. Input = 192. Output = 10.                                                                                                       
    logits = tf.layers.dense(
      inputs=dense2, units=CLASS_NUMBER, activation=tf.nn.relu, name='Logits')
    print("logits.shape {}".format(logits.shape))

    return logits

## 5. Custom TF Estimator

### 5.1 Define features columns

In [ ]:
def get_feature_columns():
  feature_columns = {
    'images': tf.feature_column.numeric_column('images', (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
  }
  return feature_columns

feature_columns = get_feature_columns()
print("Feature columns: {}".format(feature_columns))

### 5.2 TF Estimator model function

In [ ]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode, params):
    
    # Create the input layers from the features                                                                                               
    feature_columns = list(get_feature_columns().values())

    images = tf.feature_column.input_layer(features=features, feature_columns=feature_columns)
    images = tf.reshape(images, shape=(-1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    
    if FLAGS.model_name == FLAGS.lenet_model_name:
        logits = LeNet(images)
    else:
        logits = DeepLeNet(images)
    
    predictions = {
        'class_ids': tf.argmax(input=logits, axis=1, name='classes'),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor'),
        'logits': logits
    }

    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        
        export_outputs = {
            'predictions': tf.estimator.export.PredictOutput(predictions)
        }
        
        return tf.estimator.EstimatorSpec(mode, predictions=predictions, export_outputs=export_outputs)
    
    # For train and eval
    global_step = tf.train.get_or_create_global_step()
    
    # Compute the loss for both training and evaluation
    one_hot_labels = tf.one_hot(labels, CLASS_NUMBER)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot_labels, logits=logits)
    loss = tf.reduce_mean(cross_entropy)
    # Display cross_entropy metric into TensorBoard
    tf.summary.scalar('cross_entropy', loss)
    
    # Compute the accuracy.
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions['class_ids'], name='acc_op')
    metrics = {'accuracy': accuracy}
    # Display accuracy metric into TensorBoard
    tf.summary.scalar('accuracy', accuracy[1])

    # If evaluation mode, early return
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)    
    
    # Define the optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss, global_step=global_step)
    
    # From tutorial
    return tf.estimator.EstimatorSpec(
        mode, 
        loss=loss, 
        train_op=train_op, 
        predictions=predictions, 
        eval_metric_ops=metrics)

### 5.3 Serving functions

In [ ]:
def preprocess_for_serving(image):
    
    image = tf.cast(image, tf.float32)
    image = tf.image.per_image_standardization(image)
    
    return image

def serving_input_fn():
    receiver_tensor = {
        'images': tf.placeholder(shape=[None, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS], dtype=tf.float32)
    }
    features = {
        'images': tf.map_fn(preprocess_for_serving, receiver_tensor['images'])
    }
    
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

### 5.4 TF Estimator

In [ ]:
# Set Estimator configurations
run_config = tf.estimator.RunConfig(
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    #tf_random_seed=FLAGS.tf_random_seed,
    model_dir="./trained_models/{}".format(FLAGS.model_name),
    keep_checkpoint_max = FLAGS.keep_checkpoint_max
)

# Build the Estimator
classifier = tf.estimator.Estimator(
    model_fn=model_fn, 
    params={},
    config=run_config
)

### 5.5 Train and eval configurations

In [ ]:
train_spec = tf.estimator.TrainSpec(
    input_fn=generate_train_input_fn(TRAIN_TFRECORDS, batch_size=FLAGS.batch_size),
    max_steps=FLAGS.max_steps
)

# Used to export the model
exporter = tf.estimator.FinalExporter(
    name=FLAGS.exported_model_name,
    serving_input_receiver_fn=serving_input_fn,
    assets_extra=None,
    as_text=False,
)

eval_spec = tf.estimator.EvalSpec(
    input_fn=generate_eval_input_fn(EVAL_TFRECORDS, batch_size=FLAGS.batch_size),
    steps=FLAGS.eval_steps, 
    throttle_secs=FLAGS.throttle_secs,
    exporters=exporter    
)

## 6. Model training and evaluation

In [ ]:
tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

## 7. Model evaluation on test set

In [ ]:
test_classifier = tf.estimator.Estimator(model_fn=model_fn, config=run_config)
test_classifier.evaluate(input_fn=generate_eval_input_fn(TEST_TFRECORDS, FLAGS.batch_size), steps=1)